# Period Trading in a Double Auction Simulation
## Created by Alex Payne

### These files and code below will help to explain how one can use our simulation to test automated trading strategies in a continuous double auction market

In [1]:
# Hello
import spot_system as sys
import AI_Testing as prd
import random
import csv
import matplotlib.pyplot as plt
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import os
import time
import trader as tdr
import math
from timeit import default_timer as timer
import scipy.stats as stats
import spot_market_period as cda

## Below you will set where the market information and graphs will be stored on your computer

### after building a supply/demand schedule using spot_environment_gui, you will set the input_path to where your csv files are stored, and input_file to the particular csv file you wish to use

### output_path will be set to the folder you wish to dump all market information and graphs into

### session_name will be the unique identifying name you give to your market runs

In [2]:
input_path = "C:\\Users\\Summer17\\Desktop\\Repos\\DoubleAuctionMisc\\projects\\"
input_file = "TestVS"
output_path = "C:\\Users\\Summer17\\Desktop\\Repos\\DoubleAuctionMisc\\period data\\"
session_name = "Jupyter Testing"
'''Below are global dictionaries that will contain information needed to execute several functions'''
all_prices = []
theoretical_transactions = []
all_ends = []
avg_prices = []
endpoints = []
eff = []
periods_list = []
act_surplus = []
maxi_surplus = []


## Below you will set the number of periods you wish to run, the price limits, number of rounds, and the traders you wish to use 

### you can create new trading strategies by using the formatting in trader.py

In [3]:
num_periods = 10  # periods or trading days
limits = (400, 0)  # price ceiling, price floor
rounds = 25  # rounds in each period (can substitute time clock)
name = "trial"
period = 0  # ...??
try:
    os.makedirs(output_path + session_name)  # creates folder for session data
except FileExistsError:
    print("ERROR: File Exists... must rename or delete previous session data")
    raise  # raises error if folder already exists
header = session_name
smp = cda.SpotMarketPeriod(session_name, num_periods)
'''Below trader classes are abbreviated'''
zic = "Trader_ZIC"  # zero intelligence constrained
ziu = "Trader_ZIU"  # zero intelligence unconstrained trader.. not used
kp = "Trader_Kaplan"  # sniping trader based on Santa Fe paper
si = "Trader_Shaver"  # simple trader.. not used
ps = "Trader_PS"  # PS trader based on Priest and Tol paper
aa = "Trader_AA"  # aggressiveness trader based on Cliff and Vytelingum paper
gd = "Trader_GD"  # GD trader based on Gjerstadt and Dickhaut paper
#zip = "Trader_ZIP"  # zero intelligence plus trader
ai = "Trader_AI"

ERROR: File Exists... must rename or delete previous session data


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Summer17\\Desktop\\Repos\\DoubleAuctionMisc\\period data\\Jupyter Testing'

## You can set the trading strategies used by changing the trader abbreviations in trader_names

### after executing the cell you will be able to observe each buyer's and seller's individual values, equilibrium calculation, and available surplus in the market

In [ ]:
trader_names = [aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa]
header = session_name
smp.init_spot_system(name, limits, rounds, input_path, input_file, output_path, session_name)
rnd_traders = trader_names    # because shuffle shuffles the list in place, returns none
print(rnd_traders)

## Executing the cell below will start the market and initiate bids/asks and contracts until all periods have been completed

### all the information from the market will be appended to the dictionaries and also be stored, along with all the graphs, in your output path destination 

In [ ]:
for k in range(num_periods):  # iterates through number of periods or "trading days"
    periods_list.append(k)
    random.shuffle(rnd_traders)  # shuffles trader order per period
    smp.init_traders(rnd_traders, k)
    print("**** Running Period {}".format(k))  # provides visual effect in editor
    smp.run_period(period, header)
    results = smp.eval()
    '''the below data is appended into global dictionaries'''
    eff.append(results[8])  # appends the efficiencies per period
    act_surplus.append(results[7])  # appends actual surplus per period
    maxi_surplus.append(results[6])  # appends maximum surplus per period
    smp.get_contracts()  # gets transaction prices and period endpoints
    session_folder = output_path + session_name + "\\"  # establishes file path for session data folder
    smp.record_session_data(session_folder)  # records session data in excel csv

## Executing the cells below will provide the user with the market information generated by the simulation

In [ ]:
print("Market Efficiencies:" + str(eff))  # print market efficiencies
print("Avg. Efficiency:" + str(sum(eff)/num_periods))  # print avg efficiency
print("Total Avg. Transaction Price:" + str(sum(avg_prices[1:])/(num_periods - 1)))
print("Actual Surpluses:" + str(act_surplus))  # print actual surpluses
print("Maximum Surpluses:" + str(maxi_surplus))  # print max surpluses

In [ ]:
print("Strategy Total Earnings")
print("Trader_AA: " + str(smp.total_earns('AA')))
#print("Trader_AI: " + str(smp.total_earns('AI')))
print("Trader_GD: " + str(smp.total_earns('GD')))  #
print("Trader_PS: " + str(smp.total_earns('PS')))  # ADDED: section to list total avg earns
# print("Trader_AI: " + str(smp.total_avg_earns('AI')))   #
print("Trader_ZIP: " + str(smp.total_earns('ZIP')))  #
print("Trader_ZIC: " + str(smp.total_earns('ZIC')))  #
print("Trader_Kaplan: " + str(smp.total_earns('KP')))
print("Trader_Shaver: " + str(smp.total_earns('SI')))

In [ ]:
print("Strategy Total Avg. Earnings (per trader)")
print("Trader_AA: " + str(smp.total_avg_earns('AA', trader_names.count(aa)*num_periods)))   #
print("Trader_GD: " + str(smp.total_avg_earns('GD', trader_names.count(gd)*num_periods)))   #
print("Trader_PS: " + str(smp.total_avg_earns('PS', trader_names.count(ps)*num_periods)))   # ADDED: section to list total avg earns
#print("Trader_AI: " + str(smp.total_avg_earns('AI')))   #
print("Trader_ZIP: " + str(smp.total_avg_earns('ZIP', trader_names.count(zip)*num_periods))) #
print("Trader_ZIC: " + str(smp.total_avg_earns('ZIC', trader_names.count(zic)*num_periods))) #
print("Trader_Kaplan: " + str(smp.total_avg_earns('KP', trader_names.count(kp)*num_periods)))
print("Trader_Shaver: " + str(smp.total_avg_earns('SI', trader_names.count(si)*num_periods)))

## Executing the cells below will generate all the market graphs produced by the simulation 

In [ ]:
smp.get_avg_trade_ratio()  # prints avg trade ratio for all periods

In [ ]:
smp.graph_trader_eff()  # plots individual efficiency

In [ ]:
smp.graph_efficiency()  # plots period efficiency

In [ ]:
smp.get_endpoints()  # obtains endpoints of periods for graph
smp.graph_contracts()  # graphs contract transactions and avg transaction per period

In [ ]:
smp.graph_alphas()  # graphs Smith's Alpha of convergence

In [ ]:
smp.graph_distribution()  # graphs normal distribution of trader efficiencies

## Now I will demonstrate how to initiate market shocks to test trader responses to altered market information

In [ ]:
'''Reset all dictionaries'''
all_prices = []
theoretical_transactions = []
all_ends = []
avg_prices = []
endpoints = []
eff = []
periods_list = []
act_surplus = []
maxi_surplus = []

input_file_market_shock = "MarketShockTest"
new_session_name = "Jupyter Test 2"
num_periods = 10  # periods or trading days
limits = (400, 0)  # price ceiling, price floor
rounds = 25  # rounds in each period (can substitute time clock)
name = "trial"
period = 0  # ...??
try:
    os.makedirs(output_path + new_session_name)  # creates folder for session data
except FileExistsError:
    print("ERROR: File Exists... must rename or delete previous session data")
    raise  # raises error if folder already exists
smp = cda.SpotMarketPeriod(new_session_name, num_periods)
'''Below trader classes are abbreviated'''
zic = "Trader_ZIC"  # zero intelligence constrained
ziu = "Trader_ZIU"  # zero intelligence unconstrained trader.. not used
kp = "Trader_Kaplan"  # sniping trader based on Santa Fe paper
si = "Trader_Shaver"  # simple trader.. not used
ps = "Trader_PS"  # PS trader based on Priest and Tol paper
aa = "Trader_AA"  # aggressiveness trader based on Cliff and Vytelingum paper
gd = "Trader_GD"  # GD trader based on Gjerstadt and Dickhaut paper
#zip = "Trader_ZIP"  # zero intelligence plus trader
ai = "Trader_AI"
trader_names = [aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa, aa]
header = session_name
smp.init_spot_system(name, limits, rounds, input_path, input_file_market_shock, output_path, new_session_name)
rnd_traders = trader_names    # because shuffle shuffles the list in place, returns none
print(rnd_traders)

In [ ]:
for k in range(num_periods):  # iterates through number of periods or "trading days"
    if k == 3:
        smp.init_spot_system_crash(name, limits, rounds, input_path, input_file_market_shock, output_path, new_session_name)
    else:
        pass
    periods_list.append(k)
    random.shuffle(rnd_traders)  # shuffles trader order per period
    smp.init_traders(rnd_traders, k)
    print("**** Running Period {}".format(k))  # provides visual effect in editor
    smp.run_period(period, header)
    results = smp.eval()
    '''the below data is appended into global dictionaries'''
    eff.append(results[8])  # appends the efficiencies per period
    act_surplus.append(results[7])  # appends actual surplus per period
    maxi_surplus.append(results[6])  # appends maximum surplus per period
    smp.get_contracts()  # gets transaction prices and period endpoints
    session_folder = output_path + new_session_name + "\\"  # establishes file path for session data folder
    smp.record_session_data(session_folder)  # records session data in excel csv

In [ ]:
print("Market Efficiencies:" + str(eff))  # print market efficiencies
print("Avg. Efficiency:" + str(sum(eff)/num_periods))  # print avg efficiency
print("Total Avg. Transaction Price:" + str(sum(avg_prices[1:])/(num_periods - 1)))
print("Actual Surpluses:" + str(act_surplus))  # print actual surpluses
print("Maximum Surpluses:" + str(maxi_surplus))  # print max surpluses

In [ ]:
print("Strategy Total Earnings")
print("Trader_AA: " + str(smp.total_earns('AA')))
#print("Trader_AI: " + str(smp.total_earns('AI')))
print("Trader_GD: " + str(smp.total_earns('GD')))  #
print("Trader_PS: " + str(smp.total_earns('PS')))  # ADDED: section to list total avg earns
# print("Trader_AI: " + str(smp.total_avg_earns('AI')))   #
print("Trader_ZIP: " + str(smp.total_earns('ZIP')))  #
print("Trader_ZIC: " + str(smp.total_earns('ZIC')))  #
print("Trader_Kaplan: " + str(smp.total_earns('KP')))
print("Trader_Shaver: " + str(smp.total_earns('SI')))

In [ ]:
print("Strategy Total Avg. Earnings (per trader)")
print("Trader_AA: " + str(smp.total_avg_earns('AA', trader_names.count(aa)*num_periods)))   #
print("Trader_GD: " + str(smp.total_avg_earns('GD', trader_names.count(gd)*num_periods)))   #
print("Trader_PS: " + str(smp.total_avg_earns('PS', trader_names.count(ps)*num_periods)))   # ADDED: section to list total avg earns
#print("Trader_AI: " + str(smp.total_avg_earns('AI')))   #
print("Trader_ZIP: " + str(smp.total_avg_earns('ZIP', trader_names.count(zip)*num_periods))) #
print("Trader_ZIC: " + str(smp.total_avg_earns('ZIC', trader_names.count(zic)*num_periods))) #
print("Trader_Kaplan: " + str(smp.total_avg_earns('KP', trader_names.count(kp)*num_periods)))
print("Trader_Shaver: " + str(smp.total_avg_earns('SI', trader_names.count(si)*num_periods)))

In [ ]:
smp.get_avg_trade_ratio()  # prints avg trade ratio for all periods

In [ ]:
smp.graph_trader_eff()  # plots individual efficiency

In [ ]:
smp.graph_efficiency()  # plots period efficiency

In [ ]:
smp.get_endpoints()  # obtains endpoints of periods for graph
smp.graph_contracts()  # graphs contract transactions and avg transaction per period

In [ ]:
smp.graph_alphas()  # graphs Smith's Alpha of convergence

In [ ]:
smp.graph_distribution()  # graphs normal distribution of trader efficiencies